In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Open fits file and assign to variable.
datafile = fits.open('DR7_tot_6_0.fits')

In [3]:
# Inspect fits file summary to see which dataset to use by passing .info() method on the file variable.
datafile.info()

Filename: DR7_tot_6_0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    271   70841R x 127C   [I, J, I, 22A, D, D, 5E, 5E, 5E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, I, I, I, I, I, E, E, E, E, I, E, I, E, E, D, D, D, D, E, E, E, E, E, E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, I, I, I, I, I, I, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, E, E, I, E, E]   


In [4]:
# Assign header (dataset) in fits file to a variable. Inspect the column names by passing the .header method 
datahdr = datafile[1]
datahdr.header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v0.95           
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                  568 /Number of bytes per row                         
NAXIS2  =                70841 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                  127 /Number of columns in table                      
COMMENT                                                                         
COMMENT  *** Column formats ***                                                 
COMMENT                                                                         
TFORM1  = 'I       '           /                                                
TFORM2  = 'J       '        

In [5]:
# Assign data columns to variables 
westplate   = datahdr.data['plate']  
westmjd     = datahdr.data['mjd']
westfiber   = datahdr.data['fiber']
westEWHA    = datahdr.data['EWHA']
westEWHAerr = datahdr.data['EWHAERR']

In [6]:
# Entries in these columns are 16bit integer types but we want to convert them to strings in order to later assemble
# them per the sloan object id description (i.e 'spec-<plate#>-<mjd>-<fiber#>')
strplate = [] 
strmjd   = []
strfiber = []
for i in westplate:
    strplate.append(str(i))

for i in westmjd:
    strmjd.append(str(i))
    
for i in westfiber:
    strfiber.append(str(i))

In [7]:
# The object id fiber number is always expressed as a zero padded 4-digit number. Here I write a loop to pad
# the entries with zeros based on the entries length.
zfilledfiber = []
for i in strfiber:
    if len(i) == 3:
        i = '0' + i
    elif len(i) == 2:
        i = '00' + i
    elif len(i) == 1:
        i = '000' + i
    zfilledfiber.append(i)
    
#print(zfilledfiber)

['0356', '0489', '0185', '0084', '0383', '0317', '0128', '0098', '0092', '0006', '0510', '0495', '0483', '0494', '0588', '0496', '0347', '0389', '0422', '0463', '0621', '0477', '0629', '0600', '0584', '0582', '0353', '0469', '0382', '0541', '0491', '0500', '0483', '0504', '0546', '0544', '0548', '0503', '0510', '0501', '0349', '0482', '0360', '0631', '0516', '0383', '0389', '0463', '0630', '0476', '0391', '0483', '0422', '0424', '0341', '0482', '0348', '0465', '0470', '0545', '0028', '0041', '0342', '0428', '0424', '0483', '0484', '0490', '0507', '0503', '0423', '0587', '0544', '0546', '0593', '0541', '0501', '0600', '0343', '0385', '0430', '0472', '0630', '0507', '0485', '0481', '0547', '0336', '0546', '0590', '0628', '0631', '0327', '0586', '0377', '0588', '0376', '0369', '0418', '0416', '0454', '0444', '0463', '0467', '0496', '0465', '0524', '0482', '0368', '0617', '0511', '0464', '0468', '0507', '0638', '0633', '0634', '0473', '0530', '0476', '0527', '0480', '0524', '0326', '0496',

In [16]:
# multi-index for-loop to combine the column entries and reexpress in the form of the sloan object-id.
objectid = []
for i,j,k in zip(strplate,strmjd,zfilledfiber):
    objectid.append('spec-' + i + '-' + j + '-' + k)

print(objectid)

['spec-2212-53789-0356', 'spec-2212-53789-0489', 'spec-2335-53730-0185', 'spec-2340-53733-0084', 'spec-2335-53730-0383', 'spec-2783-54524-0317', 'spec-2783-54524-0128', 'spec-2783-54524-0098', 'spec-2784-54529-0092', 'spec-2784-54529-0006', 'spec-1096-52974-0510', 'spec-1096-52974-0495', 'spec-1096-52974-0483', 'spec-1096-52974-0494', 'spec-1096-52974-0588', 'spec-1096-52974-0496', 'spec-1095-52521-0347', 'spec-1095-52521-0389', 'spec-1095-52521-0422', 'spec-1095-52521-0463', 'spec-1096-52974-0621', 'spec-1095-52521-0477', 'spec-1096-52974-0629', 'spec-1095-52521-0600', 'spec-1095-52521-0584', 'spec-1095-52521-0582', 'spec-1094-52524-0353', 'spec-1094-52524-0469', 'spec-1094-52524-0382', 'spec-1094-52524-0541', 'spec-1094-52524-0491', 'spec-1094-52524-0500', 'spec-1094-52524-0483', 'spec-1094-52524-0504', 'spec-1094-52524-0546', 'spec-1094-52524-0544', 'spec-1094-52524-0548', 'spec-1094-52524-0503', 'spec-1094-52524-0510', 'spec-1094-52524-0501', 'spec-1093-52591-0349', 'spec-1094-5252

In [10]:
# Now that the singular plate,mjd,and fiber columns have been correctly formatted into object id's 
# I'll then construct a simple dataframe for the Halpha equivalent widths of the west et al 2011 Mdwarf sample
# for use in comparing my equivalent width code results against.
datframe = pd.DataFrame(index=None)
datframe['Sloan Object ID']  = objectid
datframe['West2011 EWHA']    = westEWHA
datframe['West2011 EWHAerr'] = westEWHAerr

#print(datframe)

In [11]:
dMs100test = datframe.sample(n=100)

In [15]:
sloanDLlist = dMs100test['Sloan Object ID'].to_csv('random100MsDLlist',index=False)